1. Проверить гипотезу: параметр `price` перчаток с параметром `case == Clutch Case`, в среднем сильнее зависит от показателя `wear_value`, чем с параметром `case == Glove Case`, Operation Hydra Case
2. Найти перчатки, у которых показатель `wear_value`: а) изменяется больше, чем у всех остальных; б) изменяется меньше, чем у всех остальных
3. Составить рейтинг перчаток, которые имеют сильную зависимость цены от `wear_value`, но которые можно чаще всего купить за стандартную цену. Посчитать среднюю прибыль перепродажи таких предметов за `n` дней.

In [1]:
import pandas as pd

In [37]:
df = pd.read_csv('data.csv')
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint (Well-Worn),131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave (Battle-Scarred),73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather (Battle-Scarred),94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web (Minimal Wear),609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid (Well-Worn),225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands (Well-Worn),135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands (Battle-Scarred),68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal (Field-Tested),100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega (Battle-Scarred),199.00,0.625256,1537803224,BS,Clutch Case


# Подготовка данных:

Уберем окончание значений `name`, которое указывает на `quality`:

In [ ]:
df['name'] = df['market_hash_name'][:df['market_hash_name'].str.index('(')]

Значение `price` должно быть строго больше нуля.

Исключим из выборки строки, которые не удовлетворяют данному условию, если они имеются:

In [ ]:
df = df[df['price'] > 0]
df

Значение `wear_value` должно лежать в интервале (0, 1)

Будем рассматривать только строки с корректным значением `wear_value`:

In [ ]:
df = df[(df['wear_value'] > 0) & (df['wear_value'] < 1)]
df

Уберем окончание значений `name`, которое указывает на `quality`:

In [39]:
df['name'] = df['market_hash_name'][:df['market_hash_name'].str.index('(')]

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [0        26
1        30
2        23
3        34
4        33
         ..
26236    24
26237    24
26238    24
26239    23
26240    27
Name: market_hash_name, Length: 26241, dtype: int64] of <class 'pandas.core.series.Series'>

Значение `price` должно быть строго больше нуля.

Исключим из выборки строки, которые не удовлетворяют данному условию, если они имеются:

In [35]:
df = df[df['price'] > 0]
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint (Well-Worn),131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave (Battle-Scarred),73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather (Battle-Scarred),94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web (Minimal Wear),609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid (Well-Worn),225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands (Well-Worn),135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands (Battle-Scarred),68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal (Field-Tested),100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega (Battle-Scarred),199.00,0.625256,1537803224,BS,Clutch Case


Значение `wear_value` должно лежать в интервале (0, 1)

Будем рассматривать только строки с корректным значением `wear_value`:

In [36]:
df = df[(df['wear_value'] > 0) & (df['wear_value'] < 1)]
df

,market_hash_name,price,wear_value,sold_at,quality,case
1,★ Driver Gloves | Lunar Weave (Battle-Scarred),73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather (Battle-Scarred),94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web (Minimal Wear),609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid (Well-Worn),225.99,0.382708,1559791453,WW,Clutch Case
5,★ Hand Wraps | Leather (Battle-Scarred),105.77,0.650310,1536427996,BS,"Glove Case, Operation Hydra Case"
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands (Well-Worn),135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands (Battle-Scarred),68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal (Field-Tested),100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega (Battle-Scarred),199.00,0.625256,1537803224,BS,Clutch Case
